Attempt to write a function that replicates the algoirthm X classes

In [126]:
import numpy as np
import pandas as pd
import geopandas as gp

import seaborn as sns

import time
# import matplotlib.pyplot as plt

%matplotlib inline

from collections import defaultdict
from collections.abc import Iterator
from random import shuffle, choice

In [57]:
const_pairs = pd.read_csv("../Data/const_pairs.csv")
const_tris = pd.read_csv("../Data/const_tris.csv")

In [51]:
region_name = 'North East'
df = const_pairs[const_pairs['region'] == region_name]
Y = {}
for i in range(len(df)):
    Y[df['pairing'].iloc[i]] = {df['name1'].iloc[i], df['name2'].iloc[i]}

In [159]:
region_name = 'London'
const_tris_region = const_tris[const_tris['region'] == region_name]
random_tri = const_tris_region.sample(n=1)
random_trit = random_tri[['name1','name2','name3']].values.tolist()[0]
const_pairs2 = const_pairs[(~const_pairs['name1'].isin(random_trit)) & (~const_pairs['name2'].isin(random_trit))]
df = const_pairs2[const_pairs2['region'] == region_name]
Y = {}
for i in range(len(df)):
    Y[df['pairing'].iloc[i]] = {df['name1'].iloc[i], df['name2'].iloc[i]}

In [160]:
constraints = Y
choices = defaultdict(set)
for i in constraints:
    for j in constraints[i]:
        choices[j].add(i)
unsatisfied = set(choices)
solution = []

In [161]:
def _solve(unsatisfied, choices, constraints, solution, random = True, n = 10):
    if not unsatisfied:
        # No remaining unsatisfied constraints.
        yield list(solution)
        return

    # Pick the constraint with the fewest remaining choices
    # (Knuth's "S heuristic").
#     best = min(unsatisfied, key=lambda j:len(choices[j]))
#     picked = list(choices[best])
    pick = choice(list(unsatisfied))
    picked = list(choices[pick])
    if random:
        shuffle(picked)

    # Try each choice in turn and recurse.
    for i in picked:
        _choose(unsatisfied, choices, constraints, solution, i)
        yield from _solve(unsatisfied, choices, constraints, solution)
        _unchoose(unsatisfied, choices, constraints, solution, i)

def _choose(unsatisfied, choices, constraints, solution, i):
    """Make choice i; mark constraints satisfied; and remove any
    choices that clash with it.

    """
    solution.append(i)
    for j in constraints[i]:
        unsatisfied.remove(j)
        for k in choices[j]:
            for l in constraints[k]:
                if l != j:
                    choices[l].remove(k)

def _unchoose(unsatisfied, choices, constraints, solution, i):
    """Unmake choice i; restore constraints and choices."""
    last = solution.pop()
    assert i == last
    for j in constraints[i]:
        unsatisfied.add(j)
        for k in choices[j]:
            for l in constraints[k]:
                if l != j:
                    choices[l].add(k)

In [162]:
iter = _solve(unsatisfied, choices, constraints, solution)

In [163]:
iter

<generator object _solve at 0x7f2fe3dc9cd0>

In [150]:
import time
start = time.time()
j = 0
my_dict = defaultdict(int)
for i in iter:
    j += 1
#     my_dict[str(i[0])] += 1
#     print(i)
#     if j > 25:
#         break
end = time.time()
print(f"For {j} solutions it takes {end - start:.4f}s.")

For 1287 solutions it takes 162.6353s.


In [153]:
import time
start = time.time()
j = 0
my_dict = defaultdict(int)
for i in iter:
    j += 1
    my_dict[str(i[0])] += 1
#     print(i)
#     if j > 25:
#         break
end = time.time()
print(f"For {j} solutions it takes {end - start:.4f}s.")

For 1287 solutions it takes 171.3973s.


In [164]:
next(iter)

[329,
 300,
 358,
 375,
 407,
 323,
 396,
 253,
 248,
 265,
 317,
 258,
 373,
 391,
 377,
 352,
 293,
 343,
 387,
 261,
 299,
 319,
 378,
 243,
 362,
 283,
 340,
 406,
 292,
 278,
 254,
 395,
 311,
 270,
 413]

In [165]:
next(iter)

[329,
 300,
 358,
 375,
 407,
 323,
 396,
 253,
 248,
 265,
 317,
 258,
 373,
 391,
 377,
 352,
 293,
 343,
 387,
 261,
 299,
 319,
 378,
 243,
 362,
 281,
 341,
 254,
 270,
 292,
 311,
 406,
 395,
 413,
 278]

In [175]:
j = 0
for i in iter:
    j += 1
    if j == 200000:
        break

KeyboardInterrupt: 

In [176]:
j

68482

In [ ]:
211,112

In [11]:
unsatisfied #, choices, constraints, solution, random

{'Belfast East',
 'Belfast North',
 'Belfast South',
 'Belfast West',
 'East Antrim',
 'East Londonderry',
 'Fermanagh and South Tyrone',
 'Foyle',
 'Lagan Valley',
 'Mid Ulster',
 'Newry and Armagh',
 'North Antrim',
 'North Down',
 'South Antrim',
 'South Down',
 'Strangford',
 'Upper Bann',
 'West Tyrone'}

In [13]:
constraints

{653: {'Belfast East', 'Belfast North'},
 654: {'Belfast East', 'Belfast South'},
 655: {'Belfast East', 'North Down'},
 656: {'Belfast East', 'Strangford'},
 657: {'Belfast North', 'Belfast South'},
 658: {'Belfast North', 'Belfast West'},
 659: {'Belfast North', 'East Antrim'},
 660: {'Belfast North', 'South Antrim'},
 661: {'Belfast South', 'Belfast West'},
 662: {'Belfast South', 'Lagan Valley'},
 663: {'Belfast South', 'Strangford'},
 664: {'Belfast West', 'Lagan Valley'},
 665: {'Belfast West', 'South Antrim'},
 666: {'East Antrim', 'North Antrim'},
 667: {'East Antrim', 'South Antrim'},
 668: {'East Londonderry', 'Foyle'},
 669: {'East Londonderry', 'Mid Ulster'},
 670: {'East Londonderry', 'North Antrim'},
 671: {'East Londonderry', 'West Tyrone'},
 672: {'Fermanagh and South Tyrone', 'Mid Ulster'},
 673: {'Fermanagh and South Tyrone', 'Newry and Armagh'},
 674: {'Fermanagh and South Tyrone', 'Upper Bann'},
 675: {'Fermanagh and South Tyrone', 'West Tyrone'},
 676: {'Foyle', 'W

In [18]:
for i in iter:
    print(i)

KeyError: 'Belfast North'

In [27]:
constraints

{653: {'Belfast East', 'Belfast North'},
 654: {'Belfast East', 'Belfast South'},
 655: {'Belfast East', 'North Down'},
 656: {'Belfast East', 'Strangford'},
 657: {'Belfast North', 'Belfast South'},
 658: {'Belfast North', 'Belfast West'},
 659: {'Belfast North', 'East Antrim'},
 660: {'Belfast North', 'South Antrim'},
 661: {'Belfast South', 'Belfast West'},
 662: {'Belfast South', 'Lagan Valley'},
 663: {'Belfast South', 'Strangford'},
 664: {'Belfast West', 'Lagan Valley'},
 665: {'Belfast West', 'South Antrim'},
 666: {'East Antrim', 'North Antrim'},
 667: {'East Antrim', 'South Antrim'},
 668: {'East Londonderry', 'Foyle'},
 669: {'East Londonderry', 'Mid Ulster'},
 670: {'East Londonderry', 'North Antrim'},
 671: {'East Londonderry', 'West Tyrone'},
 672: {'Fermanagh and South Tyrone', 'Mid Ulster'},
 673: {'Fermanagh and South Tyrone', 'Newry and Armagh'},
 674: {'Fermanagh and South Tyrone', 'Upper Bann'},
 675: {'Fermanagh and South Tyrone', 'West Tyrone'},
 676: {'Foyle', 'W

In [28]:
unsatisfied

{'Belfast East',
 'Belfast North',
 'Belfast South',
 'Belfast West',
 'East Antrim',
 'East Londonderry',
 'Fermanagh and South Tyrone',
 'Foyle',
 'Lagan Valley',
 'Mid Ulster',
 'Newry and Armagh',
 'North Antrim',
 'North Down',
 'South Antrim',
 'South Down',
 'Strangford',
 'Upper Bann',
 'West Tyrone'}

In [29]:
import math
math.sqrt(2)

1.4142135623730951

In [38]:
s = math.sqrt(2)
s1 = (s-1)*(10**10)
(s1 - int(s1))*(10**10)

7309513092.041016